In [ ]:
# Instruções

# 1 - É necessário baixar o arquivo players_22.csv em: 
#         https://www.kaggle.com/datasets/stefanoleone992/fifa-22-complete-player-dataset

# 2 - O dataframe df contém os dados originais do arquivo player_22.csv

# 3 - São criados outros dois dataframes principais, à partir do df, para serem aplicados aos modelos:
#          df_h --> Para o modelo hierárquico (Benzema)
#          df_kmeans --> Para o modelo não hierárquico (Messi)

# 4 - São criados dois dataframes padronizado
#          df_h_scaled --> Zscore para modelo hierárquico
#          df_kmeans_scaled --> Zscore para modelo não hierárquico

# 5 - É utilizado também a PCA para redução da dimensionalidade e para aplicar o ranking (no Kmeans)

# 6 - O código está estruturado assim:
#     DataWrangling --> EDA (Análise Exporatória dos Dados) --> Hierárquico --> Não Hierárquico 

# 7 - Se não tiver o pacote instalado tem que instalar: pip install <nomePacote>

In [ ]:
# Bibliotecas utilizadas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering, KMeans
from scipy.cluster import hierarchy
from sklearn.decomposition import PCA

In [ ]:
# Utilizado para remover os warnings
import warnings 
warnings.filterwarnings('ignore')
# Usado para plotar na célula ao invés de abrir uma nova janela
%matplotlib inline

In [ ]:
#Carregando o arquivo para df (Sem path pois está no mesmo diretório do código)
df = pd.read_csv("players_22.csv")
###### APLICANDO DATA WRANGLING ######
# Atribuindo zero em valores NA
df.fillna(0, inplace=True)
# Removendo possíveis jogadores duplicados
df.drop_duplicates(subset=['short_name'], inplace=True)
# Removendo espaços
df['short_name'] = df['short_name'].str.strip()
# Criando o df_h com jogadores na posição CF, ST para o modelo hierárquico (Benzema)
df_h = df[df['player_positions'].str.contains("CF, ST")]
# Criando o df_kmeans com jogadores na posição RW para o modelo kmeans (Messi)
df_kmeans = df[df['player_positions'].str.contains("RW")]
# Removendo colunas desnecessárias
df_h = df_h.iloc[:, [2] + list(range(37, 72))]
df_kmeans = df_kmeans.iloc[:, [2] + list(range(37, 72))]
# Preparando para nomear o índice de registros com os nomes dos jogadores 
nomes_h = df_h['short_name'].values
nomes_kmeans = df_kmeans['short_name'].values
# Removendo a coluna com os nomes dos jogadores
df_h.drop('short_name', axis=1, inplace=True)
df_kmeans.drop('short_name', axis=1, inplace=True)
# Nomeando os registros com o nome dos jogadores para rodar o modelo
df_h.index = nomes_h
df_kmeans.index = nomes_kmeans
####### FIM DO DATA WRANGLING ########

In [ ]:
# ANÁLISE EXPLORATÓRIA DOS DADOS (EDA)

In [ ]:
# Contando a quantidade de jogadores em cada posição
print(df['player_positions'].value_counts())

In [ ]:
# Quantidade de observações e variáveis nos dataframes df_h e df_kmeans
print(f'df_h ->', df_h.shape)
print(f'df_kmeans ->', df_kmeans.shape)

In [ ]:
# Printando o df_h
print(df_h)

In [ ]:
# Printando o df_kmeans
print(df_kmeans)

In [ ]:
# Estatísticas do df_h
df_h.describe()

In [ ]:
# Estatísticas do df_kmeans
df_kmeans.describe()

In [ ]:
# Correlação entre as variáveis df_h (apenas 6 primeiras variáveis)
sb.pairplot(df_h.iloc[:, :6])

In [ ]:
#Mapa de calor df_h (apenas 6 primeiras variáveis)
plt.figure(figsize=(7,7))
corr = np.corrcoef(df_h.iloc[:, :6].values, rowvar=False)
sb.heatmap(corr, annot=True, cmap='Blues', fmt='.2f', cbar=False,
           xticklabels=df_h.iloc[:, :6].columns, 
           yticklabels=df_h.iloc[:, :6].columns)

In [ ]:
# Correlação entre as variáveis df_kmeans (apenas 6 primeiras variáveis)
# Quero em vermelho para diferenciar do anterior, por isso o código adicional.

# Salva a paleta atual para restaurá-la depois
old_palette = sb.color_palette()
# Define a paleta para vermelho
sb.set_palette(sb.color_palette(['red']))
# Gera o gráfico pairplot
sb.pairplot(df_kmeans.iloc[:, :6])
# Restaura a paleta anterior
sb.set_palette(old_palette)

In [ ]:
#Mapa de calor df_kmeans (apenas 6 primeiras variáveis)
plt.figure(figsize=(7,7))
corr = np.corrcoef(df_kmeans.iloc[:, :6].values, rowvar=False)
sb.heatmap(corr, annot=True, cmap='Reds', fmt='.2f', cbar=False,
           xticklabels=df_kmeans.iloc[:, :6].columns, 
           yticklabels=df_kmeans.iloc[:, :6].columns)

In [ ]:
# Visualizando os 5 primeiros registros do df_h antes de aplicar a scala
df_h.head(5)

In [ ]:
# Aplicando a Scala no df_h em um novo dataframe chamado df_h_scaled
df_h_scaled = df_h.copy()
df_h_scaled.iloc[:, :] = StandardScaler().fit_transform(df_h)

In [ ]:
# Visualizando os 5 primeiros registros do df_h_scaled
df_h_scaled.head(5)

In [ ]:
# Visualizando os 5 primeiros registros df_kmeans antes de aplicar a scala
df_kmeans.head(5)

In [ ]:
# Aplicando a Scala no df_h em um novo dataframe chamado df_h_scaled
df_kmeans_scaled = df_kmeans.copy()
df_kmeans_scaled.iloc[:, :] = StandardScaler().fit_transform(df_kmeans_scaled)

In [ ]:
# Visualizando os 5 primeiros registros do df_h_scaled
df_kmeans_scaled.head(5)

In [ ]:
# Modelo hierárquivo (Benzemar)

In [ ]:
# Gerando o dendrograma com o df_h_scaled usando o método ward
Z = hierarchy.linkage(df_h_scaled, 'ward')
plt.figure(figsize=(18,10))
plt.grid(axis='y')
dn = hierarchy.dendrogram(Z, labels=list(df_h_scaled.index), leaf_font_size=8)
plt.grid(False)
plt.show()

In [ ]:
# Para analisar jogadores específicos, fornecendo uma lista
jogadores_especificos = ['K. Benzema', 'Anderson Talisca', 'João Félix', 'Gonçalo Ramos']
df_jogadores_especificos = df_h_scaled.loc[jogadores_especificos, :]

# Gerando um novo dendograma apenas com os jogadores específicos
Z2 = hierarchy.linkage(df_jogadores_especificos, 'ward')
plt.figure(figsize=(18,6))
plt.grid(axis='y')
dn = hierarchy.dendrogram(Z2, labels=list(df_jogadores_especificos.index), leaf_font_size=8)
plt.grid(False)
plt.show()

In [ ]:
# Gerando o dendrograma com o df_h_scaled usando o método ward
Z = hierarchy.linkage(df_h_scaled, 'ward')
plt.figure(figsize=(18,10))
plt.grid(axis='y')
dn = hierarchy.dendrogram(Z, labels=list(df_h_scaled.index), leaf_font_size=8)

plt.axhline(linestyle = '--', y=14)
plt.grid(False)
plt.show()

In [ ]:
# Applicando a clusterização para 4 grupos (Usando o cálculo da distância euclidiana e o método Ward)
n_clusters = 5
cluster = AgglomerativeClustering(n_clusters=n_clusters, affinity='euclidean', linkage='ward')
grupos = cluster.fit_predict(df_h)

In [ ]:
# Imprimindo um array com os grupos
grupos

In [ ]:
# Imprimindo o output com o resultado do agrupamento
jogadores_h = list(df_h.index)
grp_jogadores_h = {i: [] for i in range(n_clusters)}
for jogador, grupo in zip(jogadores_h, grupos):
    grp_jogadores_h[grupo].append(jogador)

for gp, ctr in grp_jogadores_h.items():
    print(f'Cluster {gp}: {ctr}\n')
    


In [ ]:
# Modelo Não Hierárquico (Messi)

In [ ]:
# Aplicando o método Elbow para estabelecer a quantidade de cluster para o Kmeans
# Será usado o df_kmeans_scaled (padronizado)
inertia = []

# Será analisada a variabilidade para 20 clusters
for k in np.arange(1, 20):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(df_kmeans_scaled)
    inertia.append(kmeans.inertia_)
    
# Plotando o gráfico do Elbow Method
plt.figure(figsize=(8, 6))
plt.plot(np.arange(1, 20), inertia, '-o')
plt.xlabel('Número de clusters')
plt.ylabel('Inertia');



In [ ]:
# Printando os labels
kmeans.labels_

In [ ]:
df_kmeans_scaled

In [ ]:
### REDUÇAO DA DIMENSIONALIDADE COM PCA PARA PLOTAR O SCATTER

# Removendo as colunas, para não ter problema com reprocessamento

# Lista das colunas a serem removidas
colunas_para_remover = ['pca1', 'pca2', 'cluster']

# Verifica se as colunas existem no DataFrame antes de tentar removê-las
colunas_existentes = [col for col in colunas_para_remover if col in df_kmeans_scaled.columns]

# Remove as colunas apenas se existirem
if colunas_existentes:
    df_kmeans_scaled = df_kmeans_scaled.drop(colunas_existentes, axis=1)
    

# Aplicando PCA para reduzir a dimensionalidade para 2 componentes principais
pca = PCA(n_components=2)
pca_components = pca.fit_transform(df_kmeans_scaled)

# Adicionando os componentes principais no dataframe df_kmeans_scaled
df_kmeans_scaled['pca1'] = pca_components[:, 0]
df_kmeans_scaled['pca2'] = pca_components[:, 1]

# Aplicando KMeans com 4 grupos nos componentes principais
kmeans = KMeans(n_clusters=4)
df_kmeans_scaled['cluster'] = kmeans.fit_predict(pca_components)

# Plotando o scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(df_kmeans_scaled['pca1'], df_kmeans_scaled['pca2'], c=df_kmeans_scaled['cluster'], cmap='viridis', s=50)
plt.title('PCA + KMeans Clusters')
plt.xlabel('pca 1')
plt.ylabel('pca 2')
plt.colorbar(label='Cluster')
plt.show()


In [ ]:
# Removendo as colunas, para não ter problema com reprocessamento

# Lista das colunas a serem removidas
colunas_para_remover = ['pca1', 'pca2', 'cluster']

# Verifica se as colunas existem no DataFrame antes de tentar removê-las
colunas_existentes = [col for col in colunas_para_remover if col in df_kmeans_scaled.columns]

# Remove as colunas apenas se existirem
if colunas_existentes:
    df_kmeans_scaled = df_kmeans_scaled.drop(colunas_existentes, axis=1)
    
# Aplicando PCA para reduzir a dimensionalidade para 2 componentes principais
pca = PCA(n_components=2)
pca_components = pca.fit_transform(df_kmeans_scaled)


# Criando um modelo com K = 4
modelo_v1 = KMeans(n_clusters = 4)
modelo_v1.fit(pca_components)

# Preparando o gráfico de área
x_min, x_max = pca_components[:, 0].min() - 5, pca_components[:, 0].max() - 1
y_min, y_max = pca_components[:, 0].min() + 1, pca_components[:, 0].max() + 5
xx, yy = np.meshgrid(np.arange(x_min, x_max, .02), np.arange(y_min, y_max, .02))
Z = modelo_v1.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Plot das áreas dos clusters
plt.figure(1)
plt.clf()
plt.imshow(Z, 
           interpolation = 'nearest',
           extent = (xx.min(), xx.max(), yy.min(), yy.max()),
           cmap = plt.cm.Paired,
           aspect = 'auto', 
           origin = 'lower')

In [ ]:
# Plot dos centróides (usa o modelo_v1 e pca_components - criados anteriormente)
plt.plot(pca_components[:, 0], pca_components[:, 1], 'k.', markersize = 4)
centroids = modelo_v1.cluster_centers_
inert = modelo_v1.inertia_
plt.scatter(centroids[:, 0], centroids[:, 1], marker = 'x', s = 169, linewidths = 3, color = 'r', zorder = 8)
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xticks(())
plt.yticks(())
plt.show()

In [ ]:
# Rodando a clusterização

# Removendo as colunas, para não ter problema com reprocessamento
# Lista das colunas a serem removidas
colunas_para_remover = ['pca1', 'pca2', 'cluster']
# Verifica se as colunas existem no DataFrame antes de tentar removê-las
colunas_existentes = [col for col in colunas_para_remover if col in df_kmeans_scaled.columns]
# Remove as colunas apenas se existirem
if colunas_existentes:
    df_kmeans_scaled = df_kmeans_scaled.drop(colunas_existentes, axis=1)
# Aplicando KMeans com 4 grupos nos componentes principais
kmeans = KMeans(n_clusters=4, random_state=22)
df_kmeans_scaled['cluster'] = kmeans.fit_predict(df_kmeans_scaled)
n_clusters = 4
grupos = df_kmeans_scaled['cluster']
#Imprimindo os agrupamentos
jogadores = list(df_kmeans_scaled.index)
grp_jogadores = {i: [] for i in range(n_clusters)}
for jogador, grupo in zip(jogadores, grupos):
    grp_jogadores[grupo].append(jogador)

for gp, ctr in grp_jogadores.items():
    print(f'Cluster {gp}: {ctr}\n')
    

In [ ]:
#Criando um ranking com os 10 jogadores no grupo do Messi usando PCA

# Procurando o número do cluster onde está o Messi
valor_especifico =  df_kmeans_scaled.loc['L. Messi', 'cluster']
# Criação do novo DataFrame com base na indexação booleana apenas com o cluster Messi
df_cluster_Messi = df_kmeans_scaled[df_kmeans_scaled['cluster'] == valor_especifico]
# Retirar a coluna cluster para aplicar a PCA
df_cluster_Messi = df_cluster_Messi.iloc[:, :-1]
# Aplicar PCA
pca = PCA(n_components=4)
pca_result = pca.fit_transform(df_cluster_Messi)
# Calcular a pontuação de cada cidade com base nos componentes principais
df_cluster_Messi['pontuacao'] = np.linalg.norm(pca_result, axis=1)
# Criar ranking dos 10 jogadores
top_jogadores = df_cluster_Messi.nlargest(10, 'pontuacao')
# Mostrar ranking 
print("Ranking cluster Messi usando PCA:")
print(top_jogadores.loc[:, ['pontuacao']])